In [1]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from string import punctuation
from os import listdir
from nltk.corpus import stopwords
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from numpy import array
from pickle import dump, load
from summa import keywords

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, load_model
from keras.utils.vis_utils import plot_model
from keras.callbacks import EarlyStopping
from keras.layers import Input, Dense, Flatten, Dropout, Embedding, Conv1D, MaxPooling1D, concatenate, Attention, Lambda
from keras import backend as K


## Develop CNN Model 

In [2]:
# load a clean dataset
def load_dataset(filename):
	return load(open(filename, 'rb'))

In [3]:
# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

In [4]:
# calculate the maximum document length
def max_length(lines):
	return max([len(s.split()) for s in lines])

In [5]:
# encode a list of lines
def encode_text(tokenizer, lines, length):
	# integer encode
	encoded = tokenizer.texts_to_sequences(lines)
	# pad encoded sequences
	padded = pad_sequences(encoded, maxlen=length, padding='post')
	return padded

In [6]:
def define_model(length, vocab_size):
    # Input layer
    inputs = Input(shape=(length,))
    embedding = Embedding(vocab_size, 100)(inputs)

    # Convolutional layer
    conv = Conv1D(filters=32, kernel_size=2, activation='relu')(embedding)
    drop = Dropout(0.2)(conv)
    pool = MaxPooling1D(pool_size=3)(drop)
    flat = Flatten()(pool)

    # Attention layer
    atten = Attention()([conv, conv])  # Use the Attention layer here

    # Attention weights
    atten_weights = Lambda(lambda x: K.mean(x, axis=1))(atten)
    
    # Merge attention weights with main output
    merged = concatenate([flat, atten_weights])

    # Dense layers
    dense1 = Dense(10, activation='relu')(merged)
    main_output = Dense(3, activation='softmax', name='main_output')(dense1)

    # Create the model
    model = Model(inputs=inputs, outputs=[main_output, atten_weights])

    # Compile
    model.compile(loss={'main_output': 'sparse_categorical_crossentropy'}, optimizer='adam', metrics=['accuracy'])

    # Summarize
    print(model.summary())
    plot_model(model, show_shapes=True, to_file='singlechannel_with_attention.png')
    
    return model

In [7]:
 # load training dataset
trainLines, trainLabels = load_dataset('../Dataset/train.pkl')
# create tokenizer
tokenizer = create_tokenizer(trainLines)
# calculate max document length
length = max_length(trainLines)
# calculate vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Max document length: %d' % length)
print('Vocabulary size: %d' % vocab_size)
# encode data
trainX = encode_text(tokenizer, trainLines, length)
print(trainX.shape)

Max document length: 3158
Vocabulary size: 11931
(13542, 3158)


In [8]:

# define model
model = define_model(length, vocab_size)

# Define early stopping callback
early_stopping = EarlyStopping(monitor='main_output_accuracy', patience=3)

# Train the model with early stopping
model.fit(trainX, array(trainLabels), epochs=15, batch_size=32,
          callbacks=[early_stopping])

# save the model
model.save('Models/cnn.h5')

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 3158)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, 3158, 100)    1193100     ['input_1[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 3157, 32)     6432        ['embedding[0][0]']              
                                                                                                  
 dropout (Dropout)              (None, 3157, 32)     0           ['conv1d[0][0]']                 
                                                                                              

In [11]:
import os

# Define the file path
file_path = '../Models/cnn.h5'

# Check if the file exists
if os.path.isfile(file_path):
    # Delete the file
    os.remove(file_path)

# Save the model
model.save(file_path)